In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/Users/ayehninnkhine/PycharmProjects/mlflow-cpe393/mlruns/1', creation_time=1742128782056, experiment_id='1', last_update_time=1742128782056, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1742128673988, experiment_id='0', last_update_time=1742128673988, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
client.create_experiment(name="my-new-experiment")

'2'

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 100",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: b0cbc283e72241b99cf27d0352dc1088, rmse: 33.9278
run id: a2e0ca290dd74cb8b025dc972fc3cf14, rmse: 33.9278
run id: 8a6422c785b543cc9e8b5199a25a3849, rmse: 33.9278
run id: 8ef77332758c4f9f841996154235b9e2, rmse: 60.1293
run id: 1b8e8e3402394bdfaa8b9a93a1042c7a, rmse: 62.8812


In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
run_id = ""  #insert your run id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

In [ ]:
#check model version and stages
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
model_version = 4
new_stage = "Staging"
# Use client.transition_model_version_stage() function to model to staging.

Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
Based on the results, update the "Production" model version accordingly.
